In [14]:
import numpy as np
import csv
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
import networkx.algorithms.community as community
import torch
import dgl
import torch.nn.functional as F
from dgl.dataloading import GraphDataLoader
from dgl.nn import GraphConv
from IPython.display import Latex
from sklearn.model_selection import train_test_split
import os.path as osp
from sklearn.metrics import roc_auc_score
import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv
from torch_geometric.utils import negative_sampling
from torch_geometric.data import DataLoader, Data
from torch.optim import Adam
import torch.nn as nn
from torch_geometric.transforms import NormalizeFeatures, RandomLinkSplit
import networkx as nx

In [34]:
test_set = pd.read_csv("../data/test_set_final.csv")
train_set = pd.read_csv("../data/train_set_final.csv")
true_test = pd.read_csv("../data/test.txt", sep=" ", header=None, names=['source', 'target'])

In [35]:
# Concatenate all the 1 to 932 column named number_source into a single column as an array of values
train_set['node_info_source'] = train_set[train_set.columns[20:952]].values.tolist()
train_set.drop(train_set.columns[20:952], axis=1, inplace=True)
test_set['node_info_source'] = test_set[test_set.columns[17:949]].values.tolist()
test_set.drop(test_set.columns[19:951], axis=1, inplace=True)


In [36]:
train_set['node_info_target'] = train_set[train_set.columns[20:952]].values.tolist()
train_set.drop(train_set.columns[20:952], axis=1, inplace=True)
test_set['node_info_target'] = test_set[test_set.columns[17:949]].values.tolist()
test_set.drop(test_set.columns[19:951], axis=1, inplace=True)



In [45]:
# Créez un graphique NetworkX
G = nx.from_pandas_edgelist(train_set, 'source', 'target', 'label')

# Convertissez le graphique en format PyG
edge_index = torch.tensor(list(G.edges)).t().contiguous()

# Créez des tenseurs pour vos étiquettes binaires
y = torch.tensor(train_set['label'].values, dtype=torch.float)

# Ajoutez des fonctionnalités supplémentaires pour chaque nœud
# Par exemple, supposons que vous ayez deux nouvelles fonctionnalités pour chaque nœud : 'feature1' et 'feature2'
x_source_features = train_set[['degree_source', 'centrality_source', 'community_source']].values
x_target_features = train_set[['degree_target', 'centrality_target', 'community_target']].values

# Concaténez les caractéristiques existantes avec les nouvelles caractéristiques
x_source_info = torch.tensor(train_set['node_info_source'].values.tolist(), dtype=torch.float)
x_target_info = torch.tensor(train_set['node_info_target'].values.tolist(), dtype=torch.float)

# Concaténez les nouvelles caractéristiques avec les caractéristiques existantes
x_source = torch.cat((x_source_info, torch.tensor(x_source_features, dtype=torch.float)), dim=1)
x_target = torch.cat((x_target_info, torch.tensor(x_target_features, dtype=torch.float)), dim=1)

x = torch.cat([x_source, x_target], dim=1)


# Créez votre jeu de données PyG avec les nouvelles fonctionnalités
data = Data(x=x, edge_index=edge_index, y=y)

# Appliquer les transformations pour diviser les données en ensembles de train, validation et test
transform = RandomLinkSplit(num_val=0.10, num_test=0.05, neg_sampling_ratio=1.0,
                            is_undirected=True, add_negative_train_samples=False)
transformed_data = transform(data)

# Après avoir appliqué la transformation, les données sont transformées en une liste de tuples (train_data, val_data, test_data)
train_data, val_data, test_data = transformed_data

print(train_data)
print(val_data)
print(test_data)


TypeError: expected Tensor as element 1 in argument 0, but got Series

In [31]:
# Créez un graphique NetworkX
G = nx.from_pandas_edgelist(test_set, 'source', 'target')

# Convertissez le graphique en format PyG
edge_index = torch.tensor(list(G.edges)).t().contiguous()


# Ajoutez des fonctionnalités supplémentaires pour chaque nœud
# Par exemple, supposons que vous ayez deux nouvelles fonctionnalités pour chaque nœud : 'feature1' et 'feature2'
x_source_features = test_set[['degree_source', 'centrality_source', 'community_source']].values
x_target_features = test_set[['degree_target', 'centrality_target', 'community_target']].values

# Concaténez les caractéristiques existantes avec les nouvelles caractéristiques
x_source_info = torch.tensor(test_set['node_info_source'].values.tolist(), dtype=torch.float)
x_target_info = torch.tensor(test_set['node_info_target'].values.tolist(), dtype=torch.float)

# Concaténez les nouvelles caractéristiques avec les caractéristiques existantes
x_source = torch.cat((x_source_info, torch.tensor(x_source_features, dtype=torch.float)), dim=1)
x_target = torch.cat((x_target_info, torch.tensor(x_target_features, dtype=torch.float)), dim=1)

x = torch.cat([x_source, x_target], dim=1)

# Supposons que num_nodes soit le nombre total de nœuds dans votre graphe
num_nodes = max(max(edge_index[0]), max(edge_index[1])) + 1

# Calculez le nombre de nœuds isolés
num_isolated_nodes = num_nodes - x.size(0)

# Créez un tensor de zéros de la taille appropriée
isolated_nodes = torch.zeros((num_isolated_nodes, x.size(1)))

# Concaténez le tensor de zéros à x
x = torch.cat([x, isolated_nodes], dim=0)

# Créez votre jeu de données PyG avec les nouvelles fonctionnalités
data_test = Data(x=x, edge_index=edge_index)


ValueError: too many dimensions 'str'

In [168]:
if data_test.edge_index.max() >= data_test.x.size(0):
    print("edge_index contains node indices that are out of bounds!")

edge_index contains node indices that are out of bounds!


In [169]:
print(data_test) 

Data(x=[3498, 1870], edge_index=[2, 3498])


In [170]:
print(data_test.edge_index.max())

tensor(7599)


In [171]:
print(data_test.x.size(0))

3498


In [172]:
(train_data.edge_index[0] <= train_data.edge_index[1]).sum()

tensor(4423)

In [173]:
data_test.validate()

ValueError: 'edge_index' contains larger indices than the number of nodes (3498) in 'Data' (found 7599)

In [174]:
class Net(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)  # Ajout d'une couche supplémentaire
        self.conv3 = GCNConv(hidden_channels, out_channels)
        self.dropout = torch.nn.Dropout(0.5)  # Ajout d'une couche de dropout

    def encode(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        x = self.dropout(x)  # Utilisation de dropout après la première couche
        x = self.conv2(x, edge_index).relu()
        x = self.dropout(x)  # Utilisation de dropout après la deuxième couche
        x = self.conv3(x, edge_index)
        return x

    def decode(self, z, edge_label_index):
        return (z[edge_label_index[0]] * z[edge_label_index[1]]).sum(dim=-1)

    def decode_all(self, z):
        prob_adj = z @ z.t()
        return (prob_adj > 0).nonzero(as_tuple=False).t()

    def forward(self, data):
        if data.edge_label_index is not None:
            z = self.encode(data.x, data.edge_index)
            return self.decode(z, data.edge_label_index)
        else:
            z = self.encode(data.x, data.edge_index)
            return self.decode_all(z)


In [175]:
# Créer votre modèle en fonction du nombre de fonctionnalités des nœuds
num_features = train_data.x.size(1)  # Nombre de fonctionnalités des nœuds source (ou target)
model = Net(num_features, 128, 64)

# Créer l'optimiseur Adam
optimizer = Adam(params=model.parameters(), lr=0.01)

# Créer la fonction de perte BCEWithLogitsLoss
criterion = nn.BCEWithLogitsLoss()

In [176]:
def train():
    model.train()
    optimizer.zero_grad()
    z = model.encode(train_data.x, train_data.edge_index)

    # We perform a new round of negative sampling for every training epoch:
    neg_edge_index = negative_sampling(
        edge_index=train_data.edge_index, num_nodes=train_data.num_nodes,
        num_neg_samples=train_data.edge_label_index.size(1))

    edge_label_index = torch.cat(
        [train_data.edge_label_index, neg_edge_index],
        dim=-1,
    )
    edge_label = torch.cat([
        train_data.edge_label,
        train_data.edge_label.new_zeros(neg_edge_index.size(1))
    ], dim=0)

    out = model.decode(z, edge_label_index).view(-1)
    loss = criterion(out, edge_label)
    loss.backward()
    optimizer.step()
    return loss


@torch.no_grad()
def test(data):
    model.eval()
    z = model.encode(data.x, data.edge_index)
    out = model.decode(z, data.edge_label_index).view(-1).sigmoid()
    return roc_auc_score(data.edge_label.cpu().numpy(), out.cpu().numpy())



In [177]:
best_val_auc = final_test_auc = 0
for epoch in range(1, 101):
    loss = train()
    val_auc = test(val_data)
    test_auc = test(test_data)
    if val_auc > best_val_auc:
        best_val = val_auc
        final_test_auc = test_auc
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Val: {val_auc:.4f}, '
          f'Test: {test_auc:.4f}')

print(f'Final Test: {final_test_auc:.4f}')

Epoch: 001, Loss: 10.4765, Val: 0.6170, Test: 0.6137
Epoch: 002, Loss: 3.7997, Val: 0.6169, Test: 0.6234
Epoch: 003, Loss: 1.2886, Val: 0.5168, Test: 0.5332
Epoch: 004, Loss: 1.0284, Val: 0.4738, Test: 0.4717
Epoch: 005, Loss: 0.8455, Val: 0.5335, Test: 0.5041
Epoch: 006, Loss: 0.7628, Val: 0.5644, Test: 0.5298
Epoch: 007, Loss: 0.7631, Val: 0.5824, Test: 0.5468
Epoch: 008, Loss: 0.7221, Val: 0.6016, Test: 0.5684
Epoch: 009, Loss: 0.7144, Val: 0.6275, Test: 0.5923
Epoch: 010, Loss: 0.7217, Val: 0.6503, Test: 0.6170
Epoch: 011, Loss: 0.6914, Val: 0.6544, Test: 0.6256
Epoch: 012, Loss: 0.6850, Val: 0.6556, Test: 0.6293
Epoch: 013, Loss: 0.6866, Val: 0.6551, Test: 0.6307
Epoch: 014, Loss: 0.6795, Val: 0.6548, Test: 0.6295
Epoch: 015, Loss: 0.6693, Val: 0.6560, Test: 0.6307
Epoch: 016, Loss: 0.6735, Val: 0.6610, Test: 0.6356
Epoch: 017, Loss: 0.6828, Val: 0.6716, Test: 0.6435
Epoch: 018, Loss: 0.6700, Val: 0.6868, Test: 0.6511
Epoch: 019, Loss: 0.6725, Val: 0.6985, Test: 0.6651
Epoch: 020,

In [178]:
len(data_test.edge_index[0])

3498

In [181]:
z = model.encode(data_test.x, data_test.edge_index)
final_edge_index = model.decode_all(z)

In [182]:
# Convertissez final_edge_index en un DataFrame
predictions_df = pd.DataFrame(final_edge_index.t().numpy(), columns=['source', 'target'])

# Ajoutez une colonne 'Predicted' avec toutes les valeurs à 1
# puisque final_edge_index contient seulement les arêtes prédites
predictions_df['Predicted'] = 1

predictions_df

,source,target,Predicted
0,0,0,1
1,0,1,1
2,0,3,1
3,0,6,1
4,0,9,1
...,...,...,...
43800507,7599,7576,1
43800508,7599,7579,1
43800509,7599,7593,1
43800510,7599,7594,1


In [183]:
true_test

,source,target
0,3425,4524
1,1620,2617
2,4832,6317
3,4984,7298
4,385,5481
...,...,...
3493,1548,2957
3494,717,1756
3495,1731,3976
3496,426,1120


In [184]:
merged_df = true_test.merge(predictions_df, on=['source', 'target'], how='left')
merged_df['Predicted'] = merged_df['Predicted'].fillna(0)


In [185]:
merged_df

,source,target,Predicted
0,3425,4524,1.0
1,1620,2617,1.0
2,4832,6317,1.0
3,4984,7298,1.0
4,385,5481,1.0
...,...,...,...
3493,1548,2957,1.0
3494,717,1756,1.0
3495,1731,3976,1.0
3496,426,1120,1.0


In [141]:
# Create a DataFrame with index and predicted labels
submission = pd.DataFrame({'ID': merged_df.index, 'Predicted': merged_df.Predicted})

submission.to_csv('../submission/predictions_gnn_3.csv', index=False)